## Understand how batch match work 

the below code is a simplified version of patch match for feature map



In [4]:
import os 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision.transforms as transforms 
import torchvision.models as models
import torchvision
import torch.nn.functional as F 
from PIL import Image 
import argparse
import copy
import math 
import numpy as np 
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt

In [35]:
style_fm = torch.rand((1, 64, 62, 64))
img_fm = torch.rand((1, 64, 62, 64))


In [36]:
    def match_fm(style_fm, img_fm):
        '''
        Input : 
            style_fm, img_fm : 1 * C * H * W

        Process:
            Instead of only compute the match for pixel inside the mask, here we compute the 
                match for the whole feature map and use mask to filter out the matched pixel we don't need 
            For feature map of size B * C * H * W, one unit of patch is by default set to be C * 3 * 3 and L2 
                Norm is computed on top of it.
            Patch Match is done in convolution fashion where content image's patch is used as kernal and applied 
                to the styled image, a score map will be computed and we construct the matched style feature map 
                on top of the score map 
            Content Feature Map is set to be unchanged and we split the Style Feature Map and create a new 
                matched style feature map 
        '''

        # Create a copy of style image fm (to matain size)
        style_fm_masked = style_fm.clone() # create a copy of the same size 
        n_patch_h = math.floor(style_fm_masked.shape[3] / 3) # use math package to avoid potential python2 issue 
        n_patch_w = math.floor(style_fm_masked.shape[2] / 3)
        print('style_fm_masked shape', style_fm_masked.shape)

        # TODO this function have not yet been tested yet
        # Use content image feature map as kernal and compute score map on style image
        stride = 3
        for i in range(n_patch_h):
            for j in range(n_patch_w):
                print('For i : {} j : {}'.format(str(i), str(j)))
                
                # Each kernal represent a patch in content image 
                kernal = img_fm[:, :, i*3:(i+1)*3, j*3:(j+1)*3]
                print('kernal size', kernal.shape)

                # For F.conv2d : https://pytorch.org/docs/stable/nn.functional.html
                score_map = F.conv2d(style_fm, kernal, stride=stride)
                score_map = score_map[0, 0, :, :]
                print('score_map shape', score_map.shape)
                print('score map is', score_map)
                
                idx = torch.argmax(score_map).item()
                matched_style_idx = (int(idx / score_map.shape[1]), int(idx % score_map.shape[1]))
                
                print('matched_style_idx {}'.format(str(matched_style_idx)))
                
                style_fm_masked[:, :, i*3:(i+1)*3, j*3:(j+1)*3] = style_fm[:, :, 
                        matched_style_idx[0]*3: (matched_style_idx[0]+1)*3, \
                            matched_style_idx[1]*3: (matched_style_idx[1]+1)*3]

        return style_fm_masked

In [37]:
out = match_fm(style_fm, img_fm)

style_fm_masked shape torch.Size([1, 64, 62, 64])
For i : 0 j : 0
kernal size torch.Size([1, 64, 3, 3])
score_map shape torch.Size([20, 21])
score map is tensor([[139.1630, 143.5382, 136.3168, 138.5848, 135.9084, 142.7011, 142.0788,
         135.6908, 144.4120, 144.7599, 144.5052, 138.2928, 143.4803, 144.4049,
         145.1765, 138.2556, 145.1522, 138.9613, 141.5054, 138.6898, 142.0926],
        [141.1291, 140.4159, 139.2292, 137.8900, 149.2051, 141.9370, 138.8234,
         146.5297, 150.8518, 148.4590, 137.3964, 144.6924, 147.6595, 130.6359,
         142.2747, 137.2898, 151.6570, 145.4133, 153.4406, 135.9830, 145.3454],
        [139.7064, 139.6488, 145.4387, 141.9807, 137.8003, 148.5686, 139.7386,
         146.2974, 143.4714, 141.2599, 139.5574, 143.0403, 145.8065, 134.1073,
         143.6881, 136.7993, 144.3113, 140.9539, 142.1877, 144.1117, 144.3156],
        [143.3705, 135.3243, 143.4717, 139.3929, 139.6627, 140.9742, 141.1595,
         140.2281, 145.9799, 149.4387, 142.1409, 137.

score map is tensor([[147.4068, 149.7589, 142.6525, 142.9885, 145.6417, 144.1820, 151.7938,
         141.7614, 144.6892, 146.3386, 149.7664, 149.4116, 147.3959, 147.7682,
         147.9104, 141.7163, 147.2363, 143.8694, 147.0167, 141.5895, 144.0538],
        [146.9186, 144.5523, 144.9862, 143.3490, 151.1457, 144.1460, 144.8027,
         148.9507, 152.8304, 152.0892, 140.4998, 147.0404, 151.5833, 137.0605,
         149.6771, 141.1218, 150.9343, 143.7800, 154.7041, 138.5254, 146.7622],
        [139.4855, 142.9852, 144.9660, 139.2459, 143.0672, 150.6541, 142.9987,
         152.1264, 146.0382, 143.7716, 149.5109, 148.9832, 151.7551, 139.2908,
         147.6709, 146.7917, 143.0509, 148.1333, 145.8282, 147.1081, 145.8938],
        [147.0237, 140.7981, 146.4346, 145.3149, 141.4888, 142.8780, 147.7814,
         145.5176, 148.3222, 148.8041, 145.6177, 145.7939, 146.6481, 144.0427,
         148.2558, 152.9640, 146.9791, 143.0976, 143.1560, 154.1743, 157.9942],
        [150.3425, 151.0438, 148.94

score map is tensor([[136.8664, 133.6527, 134.7091, 136.3875, 134.6231, 135.5425, 139.1763,
         130.8205, 139.8530, 135.0242, 142.0618, 132.4541, 136.4765, 135.3734,
         136.2750, 132.7144, 132.9836, 133.6249, 135.9586, 131.3041, 132.4041],
        [130.9189, 134.2174, 133.4126, 132.5800, 143.3335, 134.3333, 132.9641,
         141.7131, 145.3189, 138.2138, 133.3885, 137.2550, 138.5784, 128.4406,
         137.0967, 129.8468, 141.7147, 142.2196, 143.0835, 129.3598, 138.7838],
        [132.5641, 132.9834, 139.0045, 134.6226, 133.4519, 142.1467, 135.5159,
         141.1192, 135.5965, 135.3872, 134.6782, 135.8798, 141.6479, 131.5998,
         136.9321, 133.8802, 132.5691, 136.5809, 137.4082, 139.0306, 132.8916],
        [134.1262, 129.1904, 136.3096, 137.9923, 135.5303, 134.0179, 136.7772,
         135.8188, 138.9113, 142.8418, 135.7940, 139.3586, 136.2506, 137.0036,
         137.1481, 144.6866, 139.0710, 133.8977, 137.4352, 140.3680, 143.5063],
        [140.6970, 139.6460, 139.65

score map is tensor([[147.5472, 145.9006, 145.7284, 145.6192, 144.7285, 149.1000, 150.6729,
         143.5130, 150.9439, 146.4155, 153.4857, 151.8595, 148.6806, 149.6482,
         151.5013, 144.8912, 147.5682, 142.1431, 149.9226, 144.9417, 148.0371],
        [150.5413, 148.0721, 145.8705, 151.7087, 158.1686, 144.3993, 151.6900,
         152.9884, 159.4584, 152.9054, 144.0303, 144.9987, 153.9012, 138.2823,
         145.3218, 140.6524, 155.9974, 152.0886, 157.9514, 143.8534, 151.1241],
        [144.7126, 148.1801, 148.2772, 146.7406, 146.1654, 152.4568, 146.0928,
         153.4578, 149.3703, 142.0814, 149.8475, 152.8530, 152.9084, 144.0903,
         152.5591, 142.6502, 151.5011, 149.9460, 150.4760, 151.7850, 143.0931],
        [146.7915, 142.6036, 153.4778, 146.8372, 145.5725, 148.5002, 148.3612,
         148.2396, 145.6102, 155.8563, 147.6998, 145.2196, 152.4843, 146.9368,
         146.4674, 154.0003, 147.4373, 142.1881, 146.7852, 153.1635, 156.2837],
        [152.3689, 153.2855, 147.65

score_map shape torch.Size([20, 21])
score map is tensor([[147.1465, 143.5830, 138.8464, 146.6935, 141.7437, 147.8622, 146.8568,
         141.0833, 146.8216, 145.3528, 149.0626, 147.7951, 142.3671, 146.8762,
         151.6441, 143.2838, 141.8944, 140.5835, 144.3216, 138.7176, 143.3293],
        [145.9787, 146.1783, 143.5999, 143.1141, 154.7878, 143.7344, 143.2988,
         150.9510, 158.5697, 149.1982, 140.8274, 145.0873, 148.6310, 142.5391,
         145.2648, 140.6559, 150.8359, 147.9816, 154.4958, 140.9618, 144.8951],
        [141.4696, 145.2442, 150.7229, 142.0882, 140.5303, 152.7948, 141.2174,
         147.1949, 144.1983, 141.7873, 143.2999, 152.4638, 148.0501, 142.2619,
         149.5131, 144.0646, 144.7897, 146.4732, 145.5116, 146.1380, 139.6978],
        [145.4913, 141.5091, 147.8723, 147.7546, 143.7489, 143.0814, 145.9548,
         144.8865, 147.1433, 154.6269, 146.2414, 143.4156, 148.5599, 145.7418,
         147.1665, 150.1193, 147.7636, 137.8593, 146.0203, 151.0019, 150.0616]

...

score map is tensor([[138.2685, 139.0649, 136.9448, 140.5473, 140.2673, 140.5586, 142.0015,
         137.2373, 141.4838, 137.3650, 143.6712, 140.8027, 139.2210, 141.3002,
         143.9098, 135.5596, 141.6833, 133.2021, 144.0496, 132.7889, 134.7991],
        [140.8970, 140.1352, 136.5384, 137.4158, 151.2491, 137.5348, 139.7822,
         143.0624, 147.9211, 145.6192, 132.2192, 137.6890, 140.8219, 133.6029,
         141.7410, 136.4380, 146.7135, 144.1162, 147.5592, 133.2790, 138.2910],
        [134.1440, 136.8982, 141.9053, 136.5021, 140.0666, 142.6391, 138.2915,
         149.1617, 141.5589, 136.7659, 139.6606, 143.4036, 142.7672, 132.7682,
         143.0480, 141.2434, 140.6658, 139.6242, 137.0579, 144.5592, 137.2119],
        [140.4824, 135.4062, 140.5279, 143.8714, 137.8627, 140.7030, 139.4452,
         138.3583, 141.5231, 142.0767, 135.0893, 138.0635, 144.0569, 139.6719,
         139.9794, 149.6433, 140.9507, 133.6892, 140.7841, 145.5141, 148.8883],
        [144.3156, 144.4624, 142.73

score map is tensor([[144.6174, 145.3094, 142.2443, 144.1813, 143.2167, 144.6147, 150.0937,
         139.9252, 150.0127, 151.5172, 151.3379, 140.9333, 149.2288, 144.3049,
         153.1495, 142.8325, 145.2017, 137.9988, 142.7078, 141.2859, 145.2343],
        [146.0199, 146.9354, 142.0614, 143.6515, 154.3297, 146.9992, 145.4946,
         149.9382, 154.3327, 148.2309, 143.2979, 145.8495, 150.2761, 139.1583,
         145.0809, 140.6193, 154.9279, 145.3212, 155.5642, 142.5105, 147.4295],
        [141.3729, 142.9432, 149.3746, 144.4589, 144.3437, 150.3331, 149.0571,
         149.7895, 150.4888, 141.3607, 144.1260, 148.0855, 149.0061, 145.0138,
         146.6320, 146.2495, 145.3139, 143.5255, 148.1934, 146.0470, 145.3282],
        [146.3261, 142.6292, 150.7184, 145.4222, 143.8826, 145.7853, 148.5111,
         146.9729, 148.0541, 155.8715, 147.2516, 144.8581, 147.4079, 140.2971,
         147.9306, 157.0660, 149.5420, 144.3387, 148.9666, 152.4814, 156.2112],
        [149.3719, 149.8451, 144.04

score map is tensor([[144.3413, 142.8277, 141.1675, 139.3817, 141.7539, 141.9691, 145.8774,
         141.8467, 142.4554, 144.4521, 149.9305, 147.1797, 145.8913, 144.2643,
         149.4336, 140.3364, 147.3812, 141.0360, 147.0753, 136.0349, 141.5681],
        [146.3949, 143.4140, 144.3033, 140.9611, 154.5058, 141.9257, 144.3217,
         144.8517, 152.5571, 149.8445, 140.5210, 144.2930, 146.5065, 140.2488,
         143.7999, 135.8537, 152.9558, 144.4496, 152.2491, 136.8604, 146.3576],
        [143.5192, 139.4794, 145.0717, 144.4277, 137.9660, 150.1570, 143.8544,
         154.3769, 143.6666, 141.4972, 142.4147, 150.2234, 145.6004, 139.9225,
         148.0301, 145.1137, 144.9713, 140.0059, 145.1296, 145.7546, 142.6454],
        [145.4571, 136.2145, 147.5831, 144.5366, 141.3175, 138.3873, 148.7913,
         144.5644, 145.9578, 147.8979, 141.8193, 143.1669, 150.7429, 143.1226,
         148.5058, 150.6415, 142.7581, 140.4079, 146.7947, 150.7791, 152.4420],
        [144.3047, 149.1969, 145.66

score map is tensor([[140.2867, 137.3728, 135.2980, 140.1004, 132.4668, 139.8179, 142.8250,
         134.8586, 138.3672, 141.5878, 143.9848, 142.2134, 135.9191, 139.2168,
         146.1389, 140.3744, 137.6216, 137.9882, 137.8255, 134.4833, 136.9571],
        [142.2211, 138.2210, 137.9284, 135.9969, 146.2141, 136.3226, 139.1809,
         142.7401, 150.1970, 145.7905, 136.4923, 139.9437, 142.9438, 130.6650,
         138.9620, 133.1841, 144.8295, 141.5601, 150.8011, 134.2230, 141.9293],
        [134.2112, 137.1385, 144.1098, 136.2132, 136.8770, 142.5444, 140.2905,
         146.1069, 139.8534, 134.6558, 140.8791, 145.2012, 144.3450, 134.7642,
         144.7642, 143.0265, 136.4112, 137.9521, 142.3498, 140.4568, 136.8683],
        [137.1469, 135.3488, 140.9610, 138.4489, 136.4134, 139.3691, 140.5052,
         139.5725, 141.1105, 148.4510, 139.7792, 141.3292, 143.3038, 138.7807,
         143.2027, 146.5490, 139.4574, 132.4727, 140.4364, 146.0866, 149.6504],
        [139.2151, 143.0212, 141.19

RuntimeError: The expanded size of the tensor (2) must match the existing size (3) at non-singleton dimension 2.  Target sizes: [1, 64, 2, 3].  Tensor sizes: [64, 3, 3]